In [1]:
import numpy as np
from apeer_ometiff_library import io

In [2]:
filename = "/Users/ilan_gold/Downloads/linnarsson.images.ome.tif"
(array, omexml) = io.read_ometiff(filename)
array = array[0][0]

In [4]:
array = np.transpose(array[0][0], (1, 2, 0))
array = np.concatenate((array, np.zeros((array.shape[0], array.shape[1], 1), dtype=array.dtype)), axis=2)

In [5]:
from skimage.transform import pyramid_gaussian
pyramid = tuple(pyramid_gaussian(array, downscale=2, multichannel=True))
pyramid = np.asarray(list(pyramid))
pyramid = np.asarray([(255 * p).astype('uint8') for p in pyramid])

In [ ]:
import zarr
import random
import math
from numcodecs import Zlib
i = 0
tile_size = 512
for p in pyramid:
    padded = np.pad(p.transpose(2, 0, 1), pad_width = [(0, 0), (0, math.ceil(p.shape[0] / tile_size) * tile_size - p.shape[0]), (0, math.ceil(p.shape[1] / tile_size) * tile_size - p.shape[1])], constant_values=0).transpose(1,2,0)
    padded = padded.reshape(padded.shape[0]//tile_size, tile_size, padded.shape[1]//tile_size, tile_size, 3).swapaxes(1, 2)
    padded_reshape_flatten = padded.ravel()
    padded_reshape_shape = padded.shape
    del padded
    z = zarr.open(f"img_pyramid/pyramid_{i}.zarr", shape=list(padded_reshape_flatten.shape), chunks=(tile_size*tile_size*3,), compressor=Zlib(level=1), dtype='<u1')
    z.attrs['height'] = padded_reshape_shape[0]
    z.attrs['width'] = padded_reshape_shape[1]
    print(z)
    z[:,] = padded_reshape_flatten
    i = i + 1